In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [2]:
# Problem parameters
# (Double-check with Ragusa on appropriate parameters)
sigma_s = 1
sigma_t = 3
q_ext = 3
q = q_ext/2.0
# Boundary conditions
psi_left = 3
psi_right = 3

# Number of points in domain
npts = 10
# Number of cells in domain
nints = npts - 1
# Left- and right-endpoints of domain
a = 0
b = 1
cells = np.linspace(a, b, npts)
dx = np.abs(cells[1] - cells[0])

# Quadrature order
n = 32
mu_s, oots, weights = scipy.special.roots_legendre(n)

# Array to store solution values (each row corresponds to a different angle,
# and there as many columns as there are cells)
psi_vals = np.zeros((mu_s.size, nints))
# Impose BC, i.e., we know the value 
psi_vals[:, 0] = psi_left
psi_vals[:, -1] = psi_right

for mu_d, mu_idx in enumerate(mu_s):
    # Loop through cells
    for cell, cell_idx in enumerate(cells):
        cell_l = None
        cell_r = None
        # Determine what cells are to the left and right
        # of the current cell
        if mu_d > 0:
            if cell_idx == 0:
                cell_r = cell_idx + 1
            elif cell_idx == nints:
                cell_l = cell_idx - 1
            else:
                cell_l = cell_idx - 1
                cell_r = cell_idx + 1
        else:
            if cell_idx == 0:
                cell_r = cell_idx + 1
            elif cell_idx == nints:
                cell_l = cell_idx - 1
            else:
                cell_l = cell_idx - 1
                cell_r = cell_idx + 1

        # Perform forward sweep
        if cell_idx != 0 and cell_idx != nints:
            psi_vals[mu_idx, cell_idx] = (q*dx + mu_d*psi_vals[mu_idx, cell_l] - (sigma_t/2.0)*psi_vals[mu_idx, cell_l]*dx) / \
                                         (mu_d + ((sigma_t * dx)/2.0))


            

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
